# Trabalho de Implementação

## INF2912 - Otimização Combinatória
### Prof. Marcus Vinicius Soledade Poggi de Aragão
### 2015-2

### Ciro Cavani
#### BigData / Globo.com

Algoritmos de clusterização.

## Conteúdo

Esse notebook tem o desenvolvimento e avaliação do algoritmo iterativo do K-Means (algoritmo de Lloyd).

A avaliação do algoritmo é baseada em um mapeamento entre a maioria dos itens que foram atribuídos a um determinado cluster e o correspondente os valores verdadeiros gerados nesse cluster.

O K-Means teve resultados muito bons.

## Dataset

In [1]:
include("../src/clustering.jl")
import Inf2912Clustering
const Clustering = Inf2912Clustering

Inf2912Clustering

In [2]:
dataset = Clustering.dataset_tiny()
Clustering.summary(dataset)
sleep(0.2)

Number of Groups: 3
Number of Features: 16
Number of Features (group): 3
Probability of Activation: 0.8
Number of Objects (total): 100
Number of Objects per Group (min): 20
Number of Objects per Group (max): 40
Number of Objects in 1: 35
Number of Objects in 2: 32
Number of Objects in 3: 33


## K-Means

Consiste em executar o algoritmo *K-means* determinar os pontos *centrais* de cada grupo e classificar cada objeto como sendo do grupo com ponto central *mais próximo*

https://en.wikipedia.org/wiki/K-means_clustering

### Algoritmo Iterativo

1. Choose $k$ cluster centers randomly generated in a domain containing all the points,
2. Assign each point to the closest cluster center,
3. Recompute the cluster centers using the current cluster memberships,
4. If a convergence criterion is met, stop; Otherwise go to step 2.

In [3]:
"Algoritmo de clusterização K-Means (algoritmo de Lloyd)."
function kmeans(dataset, k; maxiters=20)
    inputs = map(v -> float(v[1]), dataset.data)
    
    # inicialização com amostragem sem reposição de k objetos como centros iniciais
    means = map(i -> inputs[i], randperm(length(inputs))[1:k])

    # função que calcula o índice do centro de menor distância de v
    classify(v) = indmin(map(c -> norm(c - v), means))

    assignments::Array{Int,1} = []
    iters = 0

    while iters < maxiters
        iters += 1

        # calcula o centro associado a cada objeto
        new_assignments = map(classify, inputs)

        # encerra o processamento se não tiver mudança com a última iteração
        assignments == new_assignments && break

        # recalcula os centros como a média dos pontos do último agrupamento
        assignments = new_assignments

        #println("Centros ", iters, ": ", means)
        #println("Agrupamentos ", iters, ": ", new_assignments)

        for i=1:k
            # lista todos os objetos do i-ésimo agrupamento
            i_points = map(ii -> inputs[ii], findin(assignments, i))

            isempty(i_points) && continue
            means[i] = mean(i_points)
        end
    end

    assignments
end

kmeans(dataset, 3)

100-element Array{Int64,1}:
 2
 1
 1
 3
 3
 3
 1
 3
 3
 2
 1
 3
 2
 ⋮
 1
 1
 3
 2
 1
 3
 2
 2
 3
 3
 2
 3

In [4]:
import Clustering.mapping

"Algoritmo de clusterização K-Means (algoritmo de Lloyd) \
aproximado para os grupos pré-definidos do dataset."
function kmeans_approx(dataset, k)
    assignments = kmeans(dataset, k)
    centermap = mapping(dataset, assignments, k)
    map(c -> centermap[c], assignments)
end

let
    k = dataset.groups
    @time prediction = kmeans_approx(dataset, k)
    Clustering.evaluation_summary(dataset, prediction; verbose=true)
    sleep(0.2)
end

  0.138747 seconds (140.42 k allocations: 7.028 MB, 5.72% gc time)
Matriz de Confusão:

[31 2 2
 1 27 4
 3 1 29]

Tamanho: 100
Acertos: 87
Erros: 13
Accuracy: 87.0%

Cluster 1

Tamanho: 35
Accuracy: 92.0%
Precision: 88.57%
Recall: 88.57%
F-score: 0.89

Acerto positivo: 31 (88.57%)
Acerto negativo: 61 (93.85%)
Falso negativo: 4 (30.77%)
Falso positivo: 4 (30.77%)

Cluster 2

Tamanho: 32
Accuracy: 92.0%
Precision: 90.0%
Recall: 84.38%
F-score: 0.87

Acerto positivo: 27 (84.38%)
Acerto negativo: 65 (95.59%)
Falso negativo: 5 (38.46%)
Falso positivo: 3 (23.08%)

Cluster 3

Tamanho: 33
Accuracy: 90.0%
Precision: 82.86%
Recall: 87.88%
F-score: 0.85

Acerto positivo: 29 (87.88%)
Acerto negativo: 61 (91.04%)
Falso negativo: 4 (30.77%)
Falso positivo: 6 (46.15%)



In [5]:
Clustering.test_dataset("small", kmeans_approx)
sleep(0.2)

  0.024113 seconds (58.24 k allocations: 28.869 MB, 22.74% gc time)
Matriz de Confusão:

[359 0 0
 0 344 0
 0 0 297]

Tamanho: 1000
Acertos: 1000
Erros: 0
Accuracy: 100.0%

Cluster 1

Tamanho: 359
Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F-score: 1.0

Acerto positivo: 359 (100.0%)
Acerto negativo: 641 (100.0%)
Falso negativo: 0 (NaN%)
Falso positivo: 0 (NaN%)

Cluster 2

Tamanho: 344
Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F-score: 1.0

Acerto positivo: 344 (100.0%)
Acerto negativo: 656 (100.0%)
Falso negativo: 0 (NaN%)
Falso positivo: 0 (NaN%)

Cluster 3

Tamanho: 297
Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F-score: 1.0

Acerto positivo: 297 (100.0%)
Acerto negativo: 703 (100.0%)
Falso negativo: 0 (NaN%)
Falso positivo: 0 (NaN%)



In [6]:
Clustering.test_dataset("large", kmeans_approx)
sleep(0.2)

  0.219657 seconds (448.74 k allocations: 216.615 MB, 40.77% gc time)
Matriz de Confusão:

[4086 0 0
 0 2899 0
 0 0 3015]

Tamanho: 10000
Acertos: 10000
Erros: 0
Accuracy: 100.0%

Cluster 1

Tamanho: 4086
Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F-score: 1.0

Acerto positivo: 4086 (100.0%)
Acerto negativo: 5914 (100.0%)
Falso negativo: 0 (NaN%)
Falso positivo: 0 (NaN%)

Cluster 2

Tamanho: 2899
Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F-score: 1.0

Acerto positivo: 2899 (100.0%)
Acerto negativo: 7101 (100.0%)
Falso negativo: 0 (NaN%)
Falso positivo: 0 (NaN%)

Cluster 3

Tamanho: 3015
Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F-score: 1.0

Acerto positivo: 3015 (100.0%)
Acerto negativo: 6985 (100.0%)
Falso negativo: 0 (NaN%)
Falso positivo: 0 (NaN%)

